In [14]:
!pip install keras-tuner

     |████████████████████████████████| 71kB 8.4MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp36-none-any.whl size=78939 sha256=da802efbb426c0b98f9788e2b3cafe0af0c79f91bff16fbe51a7745ec0ad1c37
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15358 sha256=6b56086f231b745c16d8172abb57be8152b14bf2f9397b3ba7cbd1be24a7b43c
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [2]:
print(tf.__version__)

2.4.0


In [3]:
fashion_mnist = keras.datasets.fashion_mnist

In [4]:
(train_images, train_labels),(test_images, test_labels) = fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [5]:
 train_images =  train_images/ train_images.max()
 test_images =  test_images/ test_images.max()

In [6]:
test_images.shape

(10000, 28, 28)

In [7]:
train_images[0]

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.    

In [8]:
train_images.shape

(60000, 28, 28)

In [9]:
train_images = train_images.reshape(len(train_images), 28, 28, 1)
test_images = test_images.reshape(len(test_images), 28, 28, 1)

In [10]:
test_images.shape

(10000, 28, 28, 1)

In [11]:
def build_model(hp):
  #hp = hyper parameters
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters = hp.Int('conv_1_filter', min_value = 64, max_value = 128, step = 16),
        kernel_size = hp.Choice('conv_1_kernel', values = [3,5]),
        activation = 'relu',
        input_shape = (28, 28, 1)
    ),

    keras.layers.Conv2D(
        filters = hp.Int('conv_2_filter', min_value = 32, max_value = 64, step = 16),
        kernel_size = hp.Choice('conv_1_kernel', values = [3,5]),
        activation = 'relu'
    ),

    keras.layers.Flatten(),
    keras.layers.Dense(
        units = hp.Int('dense_1_units', min_value = 32, max_value = 128, step = 16),
        activation = 'relu'
    ),
    keras.layers.Dense(10, activation = 'softmax')
  ])

  model.compile(optimizer = keras.optimizers.Adam(hp.Choice('learning_rate', values = [1e-2, 1e-3])),
                loss = 'sparse_categorical_crossentropy',
                metrics = ['accuracy'])
  
  return model

In [15]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [16]:
tuner_search = RandomSearch(build_model, objective='val_accuracy', max_trials= 5,
                            directory = 'output', project_name = 'Mnist_Fashion')

In [17]:
tuner_search.search(train_images, train_labels, epochs = 3, validation_split = 0.1)

Trial 5 Complete [00h 00m 21s]
val_accuracy: 0.9011666774749756

Best val_accuracy So Far: 0.9156666398048401
Total elapsed time: 00h 02m 08s
INFO:tensorflow:Oracle triggered exit


In [20]:
model = tuner_search.get_best_models(num_models = 1)[0]

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 128)       1280      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        73792     
_________________________________________________________________
flatten (Flatten)            (None, 36864)             0         
_________________________________________________________________
dense (Dense)                (None, 112)               4128880   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1130      
Total params: 4,205,082
Trainable params: 4,205,082
Non-trainable params: 0
_________________________________________________________________


In [22]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 10s 5ms/step - loss: 0.1070 - accuracy: 0.9596 - val_loss: 0.2810 - val_accuracy: 0.9137
Epoch 5/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0675 - accuracy: 0.9744 - val_loss: 0.3177 - val_accuracy: 0.9120
Epoch 6/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0398 - accuracy: 0.9861 - val_loss: 0.3548 - val_accuracy: 0.9153
Epoch 7/10
1688/1688 [==============================] - 9s 6ms/step - loss: 0.0285 - accuracy: 0.9900 - val_loss: 0.4043 - val_accuracy: 0.9132
Epoch 8/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0237 - accuracy: 0.9917 - val_loss: 0.4628 - val_accuracy: 0.9130
Epoch 9/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0159 - accuracy: 0.9945 - val_loss: 0.5352 - val_accuracy: 0.9062
Epoch 10/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0180 - accuracy: 0.9936 - val_loss: 0.5781 - val_accurac